# Importing data to CODEX tools

In [3]:
import sys
sys.path.append('/home/voehring/voehring/notebooks/codex-tools')

In [5]:
from skimage.io import imread
import codex_tools as cd

* We assume that our data comes in form of a `tiff` files. To read them into our session we use `skimage` imread function.

In [7]:
test_image = imread('../../tests/test_files/input.tiff')

* By inspecting the `shape` of our image we see that it contains 5 channels.

In [9]:
test_image.shape

(5, 500, 500)

* In order to work `codex-tools` pass the image along with a list that is descriptive for each channel of the image. In our case this looks as follows.

In [11]:
codex = cd.load_image_data(test_image, ['Hoechst', 'CD4', 'CD8', 'FOXP3', 'BCL6'])